In [ ]:
!pip install -q biopython

     |████████████████████████████████| 2.3MB 5.4MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DATA_DIR = '/content/drive/My Drive/data/ChIP-seq/'

Mounted at /content/drive


In [ ]:
!cp -r '/content/drive/My Drive/dna_nn' .
!ls dna_nn

download.py  load.py


In [ ]:
import gc

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.data import Dataset

from dna_nn.load import encode, encoded_shape, gen_from_fasta

%load_ext autoreload
%autoreload 2

# Conv2d

In [ ]:
# generators
word_size = 3
region_size = 0
subset_size = 690 * 139

def cut(x):
    return x[:60]

x_shape = encoded_shape(range(101), word_size, region_size)
encode_func = encode(word_size, region_size)

x_shape = (60 - word_size + 1, ) + x_shape[1:]

train_file = DATA_DIR + 'motif_discovery_train.fasta'
test_file = DATA_DIR + 'motif_discovery_test.fasta'
train_gen = gen_from_fasta(train_file, [cut, encode_func])
test_gen = gen_from_fasta(test_file, [cut, encode_func])

# datasets
bacth_size = 512
prefetch = tf.data.experimental.AUTOTUNE

output_shapes = (x_shape, ())
output_types = (tf.float32, tf.float32)

train_ds = Dataset.from_generator(train_gen, output_types, output_shapes)
# takes about 30 seconds to skip the training data
val_ds = train_ds.skip(subset_size).take(690 * 10)
train_ds = train_ds.take(subset_size).shuffle(500).batch(bacth_size).prefetch(prefetch)

test_ds = Dataset.from_generator(test_gen, output_types, output_shapes)
test_ds = test_ds.take(subset_size).batch(bacth_size).prefetch(prefetch)

print('x shape:', train_ds.element_spec[0].shape)
print('y shape:', train_ds.element_spec[1].shape)

x shape: (None, 58, 64, 1)
y shape: (None,)


In [ ]:
%%time
x_val, y_val = [], []
for d in val_ds:
    x_val.append(d[0])
    y_val.append(d[1])
x_val = tf.convert_to_tensor(x_val)
y_val = tf.convert_to_tensor(y_val)
validation_data = (x_val, y_val)

CPU times: user 36.1 s, sys: 2.62 s, total: 38.7 s
Wall time: 42.1 s


In [ ]:
keras.backend.clear_session()

model = keras.Sequential([
    keras.layers.Conv2D(64, 3, activation='relu', input_shape=x_shape),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(2, activation='softmax'),
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 56, 62, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 28, 31, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 28, 31, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 55552)             0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 55552)             0         
_________________________________________________________________
dense (Dense)                (None, 128)               7110784   
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 2

In [ ]:
history = model.fit(train_ds, validation_data=validation_data, epochs=15)

Epoch 1/15
188/188 [==============================] - 32s 170ms/step - loss: 0.6875 - accuracy: 0.5369 - val_loss: 0.6802 - val_accuracy: 0.5652
Epoch 2/15
188/188 [==============================] - 31s 166ms/step - loss: 0.6710 - accuracy: 0.5836 - val_loss: 0.6692 - val_accuracy: 0.5907
Epoch 3/15
188/188 [==============================] - 31s 165ms/step - loss: 0.6576 - accuracy: 0.6059 - val_loss: 0.6684 - val_accuracy: 0.5925
Epoch 4/15
188/188 [==============================] - 31s 164ms/step - loss: 0.6453 - accuracy: 0.6231 - val_loss: 0.6666 - val_accuracy: 0.5906
Epoch 5/15
188/188 [==============================] - 31s 165ms/step - loss: 0.6320 - accuracy: 0.6401 - val_loss: 0.6736 - val_accuracy: 0.5865
Epoch 6/15
188/188 [==============================] - 31s 164ms/step - loss: 0.6178 - accuracy: 0.6549 - val_loss: 0.6833 - val_accuracy: 0.5868
Epoch 7/15
188/188 [==============================] - 31s 166ms/step - loss: 0.6048 - accuracy: 0.6688 - val_loss: 0.6977 - val_ac

In [ ]:
model.evaluate(test_ds)

188/188 [==============================] - 43s 231ms/step - loss: 0.6759 - accuracy: 0.5938


[0.6759299635887146, 0.5937753915786743]